In [2]:
import h5py
import numpy as np
import scipy.io
import cv2
import os
from matplotlib import pyplot as plt

<h2>PaviaUni</h2>

In [3]:
paviaU = scipy.io.loadmat('dataset/Pavia_uni/PaviaU.mat')
npPaviaU=np.array(paviaU['paviaU'])

print ("Height:\t\t%i pixels\nWidth:\t\t%i pixels\nChannels:\t%i" % npPaviaU.shape)
print ("data-type: %s " % npPaviaU.dtype)

Height:		610 pixels
Width:		340 pixels
Channels:	103
data-type: uint16 


<h2>PaviaUni Groundtruth</h2>

In [4]:
paviaU_gt = scipy.io.loadmat('dataset/Pavia_uni/PaviaU_gt.mat')

npPaviaU_gt=np.array(paviaU_gt['paviaU_gt'])
npPaviaU_gt

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [2, 2, 2, ..., 0, 0, 0],
       [2, 2, 2, ..., 0, 0, 0],
       [2, 2, 2, ..., 0, 0, 0]], dtype=uint8)

<b>PaviaUni Groundtruth group by class</b>

In [5]:
w_dim = 610
h_dim = 340

D = { '0': 0, '1-Asphalt': 0, '2-Meadows': 0, '3-Gravel': 0, '4-Trees': 0, '5-Painted_metal_sheets': 0, '6-Bare_Soil': 0, '7-Bitumen': 0, '8-Self-Blocking_Bricks': 0, '9-Shadows': 0 }
for i in range(w_dim):
    for j in range(h_dim):
        if(npPaviaU_gt[i][j] == 0):
            D['0']+=1
        elif(npPaviaU_gt[i][j] == 1):
            D['1-Asphalt']+=1
        elif(npPaviaU_gt[i][j] == 2):
            D['2-Meadows']+=1
        elif(npPaviaU_gt[i][j] == 3):
            D['3-Gravel']+=1      
        elif(npPaviaU_gt[i][j] == 4):
            D['4-Trees']+=1
        elif(npPaviaU_gt[i][j] == 5):
            D['5-Painted_metal_sheets']+=1
            #print("5:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 6):
            D['6-Bare_Soil']+=1
            #print("6:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 7):
            D['7-Bitumen']+=1
            #print("7:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 8):
            D['8-Self-Blocking_Bricks']+=1
            #print("8:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 9):
            D['9-Shadows']+=1            
            #print("9:{x},{y}".format(x=i,y=j))  
            
D

{'0': 164624,
 '1-Asphalt': 6631,
 '2-Meadows': 18649,
 '3-Gravel': 2099,
 '4-Trees': 3064,
 '5-Painted_metal_sheets': 1345,
 '6-Bare_Soil': 5029,
 '7-Bitumen': 1330,
 '8-Self-Blocking_Bricks': 3682,
 '9-Shadows': 947}

In [53]:
6631+18649+2099+3064+1345+5029+1330+3682+947

42776

<h2>Create h5</h2>

In [6]:
def labeled(r,g,b):
    label = -1
    if(r==192 and g==192 and b==192):
        label = 0
    elif(r==0 and g==255 and b==0):
        label = 1   
    elif(r==255 and g==255 and b==0):
        label = 2  
    elif(r==0 and g==128 and b==0):
        label = 3  
    elif(r==255 and g==0 and b==255):
        label = 4  
    elif(r==41 and g==82 and b==165):
        label = 5  
    elif(r==128 and g==0 and b==128):
        label = 6  
    elif(r==0 and g==0 and b==255):
        label = 7        
    elif(r==0 and g==255 and b==255):
        label = 8  
        
    return label        

<h3>Create Train HDF5</h3>

In [111]:
#TRAIN
w_dim = 340
h_dim = 610
z_dim = 103

#Train Image
paviaU_trainImage = cv2.imread('images/pavia_university/Train_University.bmp')

textfile_train = open("/home/yavuz/caffe/data/paviaUni_new/train.txt", "w")

ii=0
for i in range(h_dim):
    for j in range(w_dim):
        
        r = paviaU_trainImage[i][j][0]
        g = paviaU_trainImage[i][j][1]
        b = paviaU_trainImage[i][j][2]
        if(r!=0 or g!=0 or b!=0):
            #print("{x},{y}".format(x=i,y=j))
            
            lbl = labeled(r,g,b) #npPaviaU_gt[i][j]
            
            samples = 1
            channels = 1
            height = 1
            width = 103
            
            h5_Train = h5py.File("/home/yavuz/caffe/data/paviaUni_new/train/train_{x}_{y}_{label}.hdf5".format(x=i,y=j,label=lbl), 'w')
            h5_Train.create_dataset('data', (samples, channels, height, width), dtype='int64')
            h5_Train.create_dataset('label', (1,1), dtype='int8')
            
            a = np.empty((samples, channels, height, width))
            for z in range(z_dim):
                a[0][0][0][z] = npPaviaU[i][j][z]
                
            h5_Train['data'][0] = a
            h5_Train['label'][0] = lbl
            
            h5_Train.close()
            
            

            textfile_train.write("/home/yavuz/caffe/data/paviaUni_new/train/train_{x}_{y}_{label}.hdf5\n".format(x=i,y=j,label=lbl))
            
            ii+=1
   
            
textfile_train.close()          
print("DONE! - {i}".format(i=ii))            

DONE! - 3921


In [8]:
w_dim = 340
h_dim = 610
z_dim = 103

samples = 1
channels = 1
height = 1
width = 103
            
            
#Train Image
paviaU_trainImage = cv2.imread('images/pavia_university/Train_University.bmp')


for i in range(h_dim):
    for j in range(w_dim):
        
        r = paviaU_trainImage[i][j][0]
        g = paviaU_trainImage[i][j][1]
        b = paviaU_trainImage[i][j][2]
        if(r!=0 or g!=0 or b!=0):
            
            
            a = np.empty((samples, channels, height, width))
            for z in range(z_dim):
                a[0][0][0][z] = npPaviaU[i][j][z]
                
            print(a)

[[[[ 1384.  1144.  1163.  1461.  1637.  1744.  1860.  1834.  1750.  1770.
     1858.  1855.  1839.  1872.  1882.  1861.  1887.  1904.  1863.  1855.
     1861.  1874.  1870.  1880.  1888.  1895.  1893.  1878.  1894.  1913.
     1902.  1881.  1860.  1817.  1808.  1826.  1840.  1851.  1859.  1845.
     1810.  1791.  1794.  1800.  1790.  1762.  1710.  1676.  1680.  1678.
     1667.  1646.  1627.  1633.  1614.  1584.  1571.  1581.  1585.  1555.
     1545.  1536.  1519.  1504.  1488.  1473.  1486.  1476.  1477.  1482.
     1461.  1434.  1417.  1392.  1365.  1375.  1392.  1385.  1363.  1353.
     1373.  1378.  1352.  1345.  1455.  1596.  1448.  1293.  1256.  1250.
     1258.  1275.  1264.  1259.  1262.  1267.  1262.  1245.  1222.  1203.
     1201.  1220.  1230.]]]]
[[[[ 1490.  1540.  1529.  1504.  1563.  1694.  1774.  1762.  1737.  1739.
     1770.  1745.  1733.  1733.  1783.  1844.  1877.  1899.  1951.  1979.
     1987.  1984.  1988.  2038.  2097.  2151.  2141.  2105.  2118.  2149.
     2174

[[[[ 1012.   717.   828.  1085.  1185.  1210.  1152.  1181.  1195.  1208.
     1177.  1112.  1158.  1186.  1203.  1260.  1251.  1211.  1206.  1196.
     1179.  1190.  1186.  1218.  1242.  1230.  1227.  1228.  1229.  1223.
     1185.  1191.  1229.  1226.  1205.  1208.  1216.  1225.  1215.  1208.
     1211.  1203.  1214.  1218.  1213.  1230.  1221.  1193.  1212.  1230.
     1245.  1252.  1241.  1255.  1261.  1257.  1264.  1264.  1251.  1250.
     1285.  1297.  1281.  1275.  1274.  1267.  1275.  1287.  1291.  1295.
     1303.  1297.  1298.  1299.  1292.  1305.  1310.  1305.  1306.  1328.
     1357.  1356.  1327.  1324.  1324.  1327.  1325.  1312.  1311.  1316.
     1352.  1372.  1327.  1312.  1310.  1316.  1319.  1316.  1323.  1347.
     1322.  1342.  1389.]]]]
[[[[ 1024.   881.   849.  1008.  1067.   998.   990.   953.   969.  1043.
     1074.  1097.  1091.  1057.  1068.  1097.  1139.  1191.  1198.  1174.
     1138.  1133.  1144.  1156.  1168.  1179.  1170.  1173.  1193.  1205.
     1224

[[[[  415.   480.   434.   388.   488.   584.   548.   507.   529.   528.
      523.   500.   473.   451.   476.   520.   512.   490.   506.   551.
      580.   594.   620.   646.   686.   720.   720.   728.   747.   765.
      780.   787.   788.   791.   799.   793.   782.   784.   788.   793.
      809.   823.   830.   832.   836.   838.   840.   851.   858.   851.
      851.   864.   880.   883.   861.   846.   860.   880.   854.   821.
      820.   826.   839.   850.   880.   944.  1042.  1149.  1253.  1334.
     1416.  1499.  1558.  1644.  1756.  1844.  1914.  1969.  2004.  2018.
     2037.  2027.  1982.  2015.  2100.  2107.  2118.  2118.  2087.  2074.
     2108.  2113.  2116.  2129.  2108.  2076.  2113.  2125.  2082.  2116.
     2201.  2222.  2180.]]]]
[[[[  641.   879.   708.   419.   487.   600.   545.   459.   409.   465.
      548.   487.   465.   487.   468.   476.   490.   508.   544.   563.
      562.   558.   592.   631.   665.   725.   748.   731.   719.   733.
      768

     1749.  1749.  1783.]]]]
[[[[  946.   764.   700.   731.   784.   851.   749.   653.   676.   680.
      644.   642.   671.   656.   616.   575.   555.   541.   531.   568.
      610.   643.   662.   662.   676.   700.   681.   672.   694.   729.
      750.   770.   771.   765.   789.   771.   755.   778.   784.   785.
      794.   806.   807.   810.   830.   837.   837.   842.   863.   878.
      879.   867.   863.   888.   899.   910.   909.   909.   913.   922.
      932.   941.   950.   953.   982.  1021.  1051.  1083.  1142.  1199.
     1232.  1247.  1287.  1364.  1402.  1425.  1473.  1521.  1541.  1564.
     1585.  1598.  1606.  1618.  1663.  1657.  1623.  1628.  1644.  1646.
     1662.  1686.  1702.  1702.  1711.  1694.  1698.  1726.  1739.  1764.
     1760.  1779.  1827.]]]]
[[[[ 1136.  1013.   833.   708.   638.   693.   728.   642.   587.   538.
      482.   471.   481.   515.   562.   547.   553.   561.   602.   636.
      625.   609.   620.   656.   659.   670.   667.  

     2098.  2144.  2114.]]]]
[[[[  606.   663.   742.   741.   755.   842.   807.   722.   658.   618.
      615.   609.   629.   601.   596.   591.   584.   614.   649.   661.
      660.   680.   701.   698.   699.   722.   755.   760.   781.   808.
      811.   826.   818.   816.   820.   829.   849.   863.   880.   888.
      888.   876.   861.   854.   870.   906.   939.   948.   933.   930.
      953.   952.   946.   966.   983.   989.   992.  1022.  1027.  1004.
     1012.  1026.  1028.  1046.  1068.  1104.  1147.  1185.  1233.  1299.
     1358.  1391.  1464.  1555.  1626.  1682.  1729.  1809.  1855.  1865.
     1889.  1916.  1887.  1894.  1969.  1985.  1970.  1993.  2036.  2079.
     2086.  2077.  2055.  2039.  2063.  2076.  2096.  2126.  2126.  2119.
     2094.  2121.  2143.]]]]
[[[[  654.   535.   538.   634.   724.   752.   642.   505.   474.   522.
      562.   558.   547.   545.   552.   577.   604.   610.   596.   594.
      599.   631.   654.   652.   680.   711.   729.  

...

     1638.  1628.  1689.]]]]
[[[[  603.   823.   911.  1033.  1131.  1082.  1036.  1068.  1135.  1161.
     1193.  1193.  1238.  1291.  1286.  1285.  1357.  1397.  1373.  1386.
     1414.  1417.  1450.  1475.  1473.  1473.  1501.  1517.  1520.  1538.
     1563.  1560.  1549.  1559.  1579.  1583.  1588.  1605.  1616.  1592.
     1578.  1592.  1612.  1625.  1637.  1637.  1619.  1624.  1645.  1639.
     1623.  1630.  1647.  1651.  1659.  1665.  1660.  1634.  1624.  1625.
     1646.  1660.  1677.  1678.  1634.  1622.  1647.  1651.  1648.  1650.
     1660.  1683.  1684.  1687.  1689.  1673.  1671.  1676.  1687.  1713.
     1721.  1718.  1733.  1710.  1710.  1713.  1712.  1717.  1721.  1698.
     1665.  1665.  1677.  1676.  1684.  1681.  1679.  1704.  1694.  1655.
     1638.  1666.  1731.]]]]
[[[[ 1275.   959.   961.  1119.  1249.  1241.  1169.  1128.  1204.  1276.
     1263.  1265.  1308.  1321.  1319.  1338.  1358.  1335.  1338.  1370.
     1418.  1460.  1485.  1458.  1456.  1484.  1491.  

     1594.  1590.  1583.]]]]
[[[[ 1501.  1158.  1071.  1350.  1383.  1245.  1219.  1126.  1097.  1188.
     1253.  1188.  1177.  1236.  1222.  1170.  1207.  1260.  1271.  1271.
     1283.  1293.  1293.  1308.  1338.  1349.  1341.  1333.  1335.  1366.
     1402.  1441.  1455.  1456.  1477.  1507.  1525.  1512.  1479.  1473.
     1486.  1483.  1488.  1534.  1559.  1555.  1561.  1562.  1549.  1562.
     1600.  1602.  1571.  1570.  1592.  1594.  1595.  1602.  1627.  1639.
     1624.  1606.  1594.  1613.  1640.  1646.  1658.  1674.  1676.  1672.
     1681.  1684.  1687.  1681.  1684.  1724.  1730.  1736.  1750.  1749.
     1761.  1770.  1751.  1754.  1784.  1791.  1764.  1725.  1748.  1772.
     1787.  1788.  1768.  1774.  1759.  1741.  1748.  1752.  1730.  1719.
     1703.  1678.  1692.]]]]
[[[[ 1351.  1117.  1220.  1189.  1126.  1179.  1237.  1280.  1235.  1248.
     1289.  1345.  1383.  1370.  1379.  1388.  1402.  1392.  1385.  1403.
     1444.  1462.  1473.  1511.  1550.  1591.  1609.  

[[[[  942.   843.   565.   200.    66.   214.   289.   237.   260.   288.
      303.   258.   236.   248.   253.   249.   263.   290.   301.   309.
      358.   424.   498.   577.   638.   680.   684.   694.   714.   758.
      786.   782.   768.   765.   754.   731.   707.   704.   700.   682.
      654.   651.   649.   642.   628.   618.   625.   609.   572.   566.
      594.   602.   594.   578.   554.   527.   520.   516.   503.   477.
      475.   478.   483.   516.   571.   661.   785.   959.  1166.  1373.
     1558.  1720.  1897.  2090.  2235.  2331.  2431.  2528.  2602.  2650.
     2688.  2729.  2744.  2705.  2699.  2724.  2751.  2749.  2752.  2770.
     2784.  2781.  2756.  2751.  2759.  2755.  2754.  2786.  2771.  2730.
     2752.  2764.  2769.]]]]
[[[[ 1270.  1415.  1440.  1307.  1274.  1318.  1407.  1440.  1413.  1384.
     1401.  1431.  1452.  1459.  1462.  1484.  1506.  1530.  1519.  1525.
     1529.  1538.  1536.  1514.  1522.  1573.  1595.  1620.  1638.  1639.
     1632

[[[[ 1340.  1076.   797.   849.   921.   936.   970.   952.   918.   967.
      942.   912.   972.  1016.   973.   959.   998.  1022.  1022.  1024.
     1032.  1039.  1045.  1084.  1150.  1173.  1165.  1155.  1157.  1186.
     1223.  1210.  1196.  1181.  1170.  1183.  1193.  1166.  1149.  1142.
     1145.  1138.  1113.  1094.  1088.  1073.  1066.  1073.  1076.  1049.
     1040.  1056.  1039.  1005.   958.   933.   920.   912.   905.   914.
      909.   891.   889.   887.   892.   919.   982.  1107.  1295.  1517.
     1720.  1915.  2183.  2484.  2786.  3107.  3381.  3626.  3827.  3997.
     4099.  4148.  4143.  4118.  4257.  4323.  4371.  4397.  4402.  4428.
     4456.  4461.  4453.  4452.  4462.  4397.  4323.  4302.  4306.  4307.
     4334.  4360.  4386.]]]]
[[[[  868.   751.   604.   534.   491.   457.   527.   608.   619.   622.
      637.   536.   424.   404.   444.   506.   559.   575.   545.   524.
      549.   582.   625.   689.   764.   810.   846.   856.   867.   888.
      903

     2734.  2754.  2767.]]]]
[[[[  966.   730.   579.   671.   772.   728.   722.   788.   850.   840.
      817.   811.   847.   861.   850.   830.   863.   915.   936.   934.
      941.   954.   975.  1013.  1067.  1101.  1123.  1157.  1178.  1217.
     1250.  1275.  1273.  1276.  1308.  1312.  1299.  1296.  1293.  1305.
     1305.  1306.  1316.  1328.  1338.  1357.  1369.  1379.  1380.  1363.
     1356.  1362.  1370.  1367.  1363.  1354.  1367.  1381.  1380.  1370.
     1369.  1361.  1371.  1395.  1430.  1464.  1538.  1645.  1752.  1852.
     1952.  2043.  2142.  2234.  2328.  2478.  2571.  2627.  2661.  2707.
     2758.  2801.  2783.  2762.  2821.  2858.  2851.  2826.  2817.  2853.
     2893.  2881.  2863.  2862.  2880.  2849.  2792.  2816.  2858.  2851.
     2818.  2808.  2791.]]]]
[[[[  820.   729.   849.   870.   843.   890.   861.   852.   793.   702.
      699.   695.   727.   769.   785.   805.   815.   850.   861.   869.
      891.   907.   956.   991.  1012.  1051.  1082.  

In [112]:
test = h5py.File("/home/yavuz/caffe/data/paviaUni_new/train/train_455_10_2.hdf5",'r')
test['data']

<HDF5 dataset "data": shape (1, 1, 1, 103), type "<i8">

In [106]:
test['data'][0]

array([[ 767,  854, 1091, 1097, 1122, 1156, 1100, 1071, 1059, 1102, 1062,
        1006, 1000, 1005, 1030, 1056, 1096, 1086, 1099, 1134, 1150, 1146,
        1155, 1166, 1192, 1243, 1255, 1249, 1274, 1301, 1321, 1328, 1354,
        1386, 1387, 1397, 1399, 1399, 1409, 1409, 1424, 1445, 1455, 1455,
        1452, 1452, 1442, 1441, 1449, 1459, 1496, 1503, 1471, 1454, 1434,
        1431, 1448, 1486, 1495, 1475, 1475, 1476, 1478, 1478, 1462, 1460,
        1481, 1497, 1522, 1523, 1507, 1488, 1487, 1483, 1497, 1533, 1552,
        1561, 1555, 1562, 1543, 1491, 1436, 1462, 1562, 1584, 1537, 1489,
        1481, 1498, 1535, 1545, 1530, 1498, 1478, 1463, 1467, 1495, 1512,
        1524, 1496, 1491, 1463]])

<h3>Create Test HDF5</h3>

In [113]:
#TEST
w_dim = 340
h_dim = 610
z_dim = 103

#TEST Image
paviaU_testImage = cv2.imread('images/pavia_university/Test_University.bmp')

textfile_test = open("/home/yavuz/caffe/data/paviaUni_new/test.txt", "w")

ii=0
for i in range(h_dim):
    for j in range(w_dim):
        
        r = paviaU_testImage[i][j][0]
        g = paviaU_testImage[i][j][1]
        b = paviaU_testImage[i][j][2]
        if(r!=0 or g!=0 or b!=0):
            #print("{x},{y}".format(x=i,y=j))
            
            lbl = labeled(r,g,b)
            
            samples = 1
            channels = 1
            height = 1
            width = 103
            
            h5_Train = h5py.File("/home/yavuz/caffe/data/paviaUni_new/test/test_{x}_{y}_{label}.hdf5".format(x=i,y=j,label=lbl), 'w')
            h5_Train.create_dataset('data', (samples, channels, height, width), dtype='int64')
            h5_Train.create_dataset('label', (1,1), dtype='int8')
            
            a = np.empty((samples, channels, height, width))
            for z in range(z_dim):
                a[0][0][0][z] = npPaviaU[i][j][z]
                
            h5_Train['data'][0] = a
            h5_Train['label'][0] = lbl
            
            h5_Train.close()
            
            

            textfile_test.write("/home/yavuz/caffe/data/paviaUni_new/test/test_{x}_{y}_{label}.hdf5\n".format(x=i,y=j,label=lbl))
            
            ii+=1
   
            
textfile_test.close()          
print("DONE! - {i}".format(i=ii))     

DONE! - 42776
